# New Features in v0.1.0

This notebook demonstrates the new features added in the v0.1.0 release:
- New Screeners (Bond, Futures, Coin)
- Fluent API (select, where)
- Field Discovery (search, technicals)
- Field Presets
- Time Interval Support

In [ ]:
%load_ext autoreload
%autoreload 2

## New Screeners

In addition to Stock, Forex, and Crypto screeners, we now have Bond, Futures, and Coin screeners.

In [ ]:
from tvscreener import BondScreener, BondField

bs = BondScreener()
df = bs.get()
print(f"Bond screener returned {len(df)} rows with {len(df.columns)} columns")
df.head()

In [ ]:
from tvscreener import FuturesScreener, FuturesField

fs = FuturesScreener()
df = fs.get()
print(f"Futures screener returned {len(df)} rows with {len(df.columns)} columns")
df.head()

In [ ]:
from tvscreener import CoinScreener, CoinField

cs = CoinScreener()
df = cs.get()
print(f"Coin screener returned {len(df)} rows with {len(df.columns)} columns")
df.head()

## Fluent API

Use `select()` and `where()` for cleaner, more readable code.

In [ ]:
from tvscreener import StockScreener, StockField, FilterOperator

ss = StockScreener()

# Select specific fields
ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.CHANGE_PERCENT,
    StockField.VOLUME,
    StockField.MARKET_CAPITALIZATION,
    StockField.RELATIVE_STRENGTH_INDEX_14
)

# Add filters using where()
ss.where(StockField.MARKET_CAPITALIZATION, FilterOperator.ABOVE, 1e9)
ss.where(StockField.VOLUME, FilterOperator.ABOVE, 1000000)

df = ss.get()
print(f"Found {len(df)} large-cap, high-volume stocks")
df.head(10)

## Field Discovery

With 13,000+ fields, you need tools to find what you're looking for.

In [ ]:
from tvscreener import StockField, CryptoField, ForexField, BondField, FuturesField, CoinField

print("Field counts by screener type:")
print(f"  StockField:   {len(list(StockField))} fields")
print(f"  CryptoField:  {len(list(CryptoField))} fields")
print(f"  ForexField:   {len(list(ForexField))} fields")
print(f"  BondField:    {len(list(BondField))} fields")
print(f"  FuturesField: {len(list(FuturesField))} fields")
print(f"  CoinField:    {len(list(CoinField))} fields")

In [ ]:
# Search for fields by name or label
rsi_fields = StockField.search("rsi")
print(f"Found {len(rsi_fields)} RSI-related fields:")
for f in rsi_fields[:10]:
    print(f"  {f.name}: {f.label}")
print(f"  ... and {len(rsi_fields) - 10} more")

In [ ]:
# Search for MACD fields
macd_fields = StockField.search("macd")
print(f"Found {len(macd_fields)} MACD-related fields:")
for f in macd_fields[:10]:
    print(f"  {f.name}: {f.label}")

In [ ]:
# Get all technical indicator fields (those supporting time intervals)
technicals = StockField.technicals()
print(f"Found {len(technicals)} technical indicator fields")
print("\nExamples:")
for f in technicals[:5]:
    print(f"  {f.name}: {f.label}")

In [ ]:
# Get recommendation fields
recos = StockField.recommendations()
print(f"Found {len(recos)} recommendation fields:")
for f in recos[:10]:
    print(f"  {f.name}: {f.label}")

## Field Presets

Use curated field groups for common analysis needs.

In [ ]:
from tvscreener import list_presets, get_preset

# List all available presets
presets = list_presets()
print(f"Available presets ({len(presets)}):")
for p in presets:
    print(f"  {p}")

In [ ]:
from tvscreener import StockScreener, STOCK_VALUATION_FIELDS, STOCK_DIVIDEND_FIELDS

ss = StockScreener()

# Combine presets for value/dividend analysis
ss.specific_fields = STOCK_VALUATION_FIELDS + STOCK_DIVIDEND_FIELDS

df = ss.get()
print(f"Valuation + Dividend analysis with {len(df.columns)} columns")
df.head()

In [ ]:
# Get preset by name
perf_fields = get_preset('stock_performance')
print(f"stock_performance preset has {len(perf_fields)} fields:")
for f in perf_fields:
    print(f"  {f.label}")

In [ ]:
from tvscreener import CryptoScreener, CRYPTO_TECHNICAL_FIELDS

cs = CryptoScreener()
cs.specific_fields = CRYPTO_TECHNICAL_FIELDS

df = cs.get()
print(f"Crypto technical analysis with {len(df.columns)} columns")
df.head()

## Time Intervals for Technical Fields

Technical indicators can be requested with different time intervals.

In [ ]:
from tvscreener import StockScreener, StockField

ss = StockScreener()

# Get RSI on different timeframes
rsi_1h = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("60")   # 1-hour
rsi_4h = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("240")  # 4-hour
rsi_1d = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("1D")   # Daily

print(f"RSI 1h field_name: {rsi_1h.field_name}")
print(f"RSI 4h field_name: {rsi_4h.field_name}")
print(f"RSI 1D field_name: {rsi_1d.field_name}")

In [ ]:
from tvscreener import StockScreener, StockField

ss = StockScreener()

# Multi-timeframe RSI analysis
ss.specific_fields = [
    StockField.NAME,
    StockField.PRICE,
    StockField.RELATIVE_STRENGTH_INDEX_14,  # Default timeframe
    StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("60"),   # 1-hour
    StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("240"),  # 4-hour
]

df = ss.get()
df.head(10)

## Type-Safe Validation

The library now validates that you're using the correct field types with each screener.

In [ ]:
from tvscreener import StockScreener, ForexField, FilterOperator

ss = StockScreener()

# This will raise a TypeError - using ForexField with StockScreener
try:
    ss.add_filter(ForexField.NAME, FilterOperator.EQUAL, "test")
except TypeError as e:
    print(f"TypeError caught: {e}")

## Summary

The v0.1.0 release adds:

| Feature | Description |
|---------|-------------|
| New Screeners | BondScreener, FuturesScreener, CoinScreener |
| Fluent API | `select()` and `where()` methods for cleaner code |
| Field Discovery | `search()`, `technicals()`, `recommendations()` |
| Field Presets | 23 curated field groups for common use cases |
| Time Intervals | `with_interval()` for technical indicators |
| Type Validation | Catches field/screener mismatches early |